# A broad light in a world full of petrol

When residential solar energy equipment was first made available to homeowners it was costly. The people who made the switch were usually those who wanted to make a conscious effort to reduce their carbon footprints. But they also had the means to invest in the equipment and services needed to make that switch. In the beginning, like many other new technologies, initial adopters often paid more until the technologies become more mainstream.

Yes, solar energy is increasingly becoming a viable fuel source for everyone. Today, switching to solar energy is far more affordable with the help of programs like the Investment Tax Credit (ITC). This incentive provides a tax credit of 26 percent of the cost to install solar power at your home. While this credit is in effect at 26% until the end of 2022, once it expires, solar energy will still remain a low-priced source of power as prices will most likely continue to drop. Meanwhile, oil, gas, and coal prices are likely to continue to increase, especially as sources are depleted and the costs to obtain these fuels grow.

In the following report, we'll explore the alternate solution to the energy problem which is the solar market, we'll research it's growth and it's spread globally in the past years, to see how the world is running into the inevitable shifting to using solar energy as the planet's main resource.

*Note: The data we will use contains the initials 'GEM', which stands for Global Energy Monitor, an organization that monitors the global energy status around the world. The data you'll see was taken straightly from the GEM data sources.*

<div style="border:solid green 2px; padding: 5px"> <h1 style="color:green; margin-bottom:20px">Note for the reader!</h1>
<p style="font-size: 16px"><strong>It is strongly and highly recommended to run the <u>entire</u> Jupyther Notebook from the beginning</strong>, as the missing values handling is operating mapping processes using coordinates, and the whole processes combined take averagely 75:16 minutes (1:15 hrs) to run.</p>

---------------
---------------

# 1. Initialization

We'll start by importing the necessary libraries for the research ahead:

In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as st
import scipy.interpolate as intp
import math as mt
import random as rd
import re
import requests
import time
from time import perf_counter
from IPython.display import display_html
from itertools import chain,cycle

# Geo-Mapping libraries:
try: # importing geopy
    import geopy.geocoders as gc
    from geopy.geocoders import Nominatim
    from geopy.extra.rate_limiter import RateLimiter
    from geopy.point import Point
except: # installing geopy using pip (in case environment lack the library)
    !pip install geopy
    import geopy.geocoders as gc
    from geopy.geocoders import Nominatim
    from geopy.extra.rate_limiter import RateLimiter
    from geopy.point import Point

# Translations libraries & requirements for the libraries:
try: # installing translators
    import translators as ts
except: # installing translators using pip (in case environment lack the library)
    ! pip install translators --upgrade
    import translators as ts
try: # installing loguru
    import loguru
except: # installing loguru using pip (in case environment lack the library)
    ! pip install loguru
    import loguru
try: # installing pathos
    import pathos
except: # installing pathos using pip (in case environment lack the library)
    ! pip install pathos
    import pathos

# Data visualizations libraries:
import matplotlib.pyplot as plt
import seaborn as sns
try: # importing missingno
    import missingno as msno
except: # installing missingno using pip (in case environment lack the library)
    !pip install missingno
    import missingno
try: # importing plotly
    import plotly.express as px
    import plotly.graph_objects as go
    import plotly.figure_factory as ff
except: # installing plotly using pip (in case environment lack the library)
    !pip install plotly-express
    import plotly.express as px
    import plotly.graph_objects as go
    import plotly.figure_factory as ff

Creating an index slicing variable:

In [ ]:
# Creating an Index Slice variable -
idx = pd.IndexSlice

### 1.1 Load Data

Now the raw data will be loaded into a DataFrame variable:

In [ ]:
# A) Loading the data we'll use for analysis -
global_solar = pd.read_excel("C:\DataBases\CBridge\Global-Solar-Power-Tracker-May-2022.xlsx", \
                                 sheet_name='Data')

# B) For the sake of simplicity, these following DFs 
# will represent the tables from the 'About' section in the original xlsx file of the DataBase:
# 1. Columns data -
cols_data = pd.read_excel("C:\DataBases\CBridge\Global-Solar-Power-Tracker-May-2022.xlsx", \
                          sheet_name='Columns')
# 2. definition of each status value in the `status` column -
status_def = pd.read_excel("C:\DataBases\CBridge\Global-Solar-Power-Tracker-May-2022.xlsx", \
                           sheet_name='Status_Definition')
# 3. Solar farms capacity cut-off values in MWs -
cutoff_thresholds = pd.read_excel("C:\DataBases\CBridge\Global-Solar-Power-Tracker-May-2022.xlsx", \
                           sheet_name='MW_Cutoff_Thresholds')

Since data is expected to be pretty big and include a big amount of columns, I'll set the print options to print *all* columns in every time we'll display a DataFrame:

In [ ]:
# Setting the DF printing to print all columns in each DF we'll print -
pd.set_option('display.max_columns', None)

### 1.2 Introduction to the data

At the current section, we'll study the data from the 'About' section of the xlsx file we imported the data from.

#### A) Columns coverage

The raw data contains the following columns:
- **`Country`** - Country the project is located in (Unabbreviated)


- **`Project Name`** - Commonly used name.
    - **Additional Note:** Preference is for a government provided name. Without that information, the name the owner or operating company uses was taken.


- **`Phase name`** - Common name of the project phase.
    - **Additional Note:** Roman numerals are changed to numbers. If the Owner uses non-sequential numbers or other characters, those are used.


- **`Project Name in Local Language / Script`** - Commonly used name in the locally used script.
    - **Additional Note:** Left blank for local languages that use the roman/latin alphabet. Otherwise local name in local script provided if possible


- **`Other plant names`** - Alternative names, including the name in local script.


- **`Capacity` (MW)** - Nameplate capacity, or best available data, of the phase in megawatts rounded to whole number.
    - **Additional Note:** Solar farm capacity cut-off, Arab Countries: <10 MW not included; Rest of world: <20 MW not included


- **`Capacity Rating` (MWac, MWp/dc, unknown)** - The reported capacity rating, unknown selected if not provided.
    - **Additional Note:** Capacity may be reported as the DC / peak value the arrays could provide or as the AC value of what will be tied into the grid. This is often not specified and could be either value when reported as just 'MW'


- **`Status`** - *see section below*.


- **`Start year`** - Year the project is or is expected to be commissioned.
    - **Additional Note:** Latest date was taken when original data source provided a range; Start years not included for projects that are canceled or shelved


- **`Retired year`** - Year the plant is taken offline.
    - **Additional Note:** This includes mothballed plants - whenever the unit stops operating.


- **`Operator`** - The company that operates the project.
    - **Additional Note:** Company that performs day-to-day operations and maintenance of the project


- **`Operator Name in Local Language / Script`** - Company name that operates the project in the locally used script.
    - **Additional Note:** Left blank for local languages use the roman/latin alphabet. Otherwise operator name in local script included if possible


- **`Owner`** - The company that directly owns the plant; When owned by a special purpose vehicle, the next level of ownership is provided.


- **`Owner Name in Local Language / Script`** - Company name that directly owns the project in the locally used script.
    - **Additional Note:** Left blank for local languages that use the roman/latin alphabet. Otherwise direct owner name included in local script if possible.


- **`Latitude` (decimal degrees)** - Latitude, assumed WGS84 (Google maps), of the project (not by unit).


- **`Longitude` (decimal degrees)** - Longitude, assumed WGS84 (Google maps), of the project (not by unit).


- **`Location accuracy` (approximate, exact)** - Location is Approximate based on using nearest city or other information in data sources, or Location is Exact when explicitly provided in data source.


- **`City`** - City, town, village, or township (China) where project is located.


- **`Local Area`** - County, taluk, or district (China) where project is located .


- **`Major Area`** - Prefecture, district, or municipality (China) where project is located.


- **`State/Province`** - Subnational unit where project is located.


- **`Region`** - Region location based on IRENA definition.


- **`GEM Location ID` (L8+5 digits)** - GEM generated ID unique for each project.


- **`GEM Phase ID` (G8+5 digits)** - GEM generated ID unique for each phase.


- `Other IDs (location)`


- `Other IDs (unit/phase)`


- **`Wiki URL`** - URL of existing or planned GEM.wiki page.

#### B) Regarding the `Status` column - these are the definitions of each project's status:

- ***The 'operating' status:*** 

Commercial operation date achieved.

- ***The 'construction' status:*** 

Equipment installation has begun (not just clearing/roads)

- ***The 'development' status:*** 

Projects that are actively moving forward in seeking governmental approvals, land rights, or financing.

- ***The 'announced' status:*** 

Projects that been publicly reported but have not yet moved actively forward by applying for permits or seeking land, material, or financing. Examples: (1) projects are the potential “Phase 2” at a location where “Phase 1” is currently under development, (2) Projects that are described in long-range company or governmental planning documents.

- ***The 'shelved' status:*** 

Suspension announced, or 2 years with no published updates

- ***The 'mothballed' status:*** 

Disused but not dismantled

- ***The 'retired' status:*** 

Dismantled

- ***The 'canceled' status:*** 

Cancellation announced, or 4 years with no published updates

#### C) A note regarding the solar farms capacity cutoff voltage

Below are added the capacity cutoff voltages values (the minimum voltage) in **Arab countries, which is any capacity under 10 MWs**, while **the rest of the world cutoff voltage capacity is under 20 MWs**.

In [ ]:
# Reminder of the minimum capacity cutoff voltages stats -
display(cutoff_thresholds)

***The next steps will be thus:***
1. We'll explore the initial data in order to find defects, missing values, duplicates etc. Since we already know the columns are all capitalized, these will be transformed in the Data Preprocessing section.
2. We'll preprocess all the data to fix all possible problems we will find during the Data Exploration section.
3. Analysis: We'll visualize the data in order to analyze the energy output, average output, etc.

---------------

# 2. Exploring the data

### 2.1 General exploration

Let's begin exploring the `global_solar` DF by having a glance at the first rows of the data:

In [ ]:
# Printing the first 3 rows of the data -
display(global_solar.head(3))

And the last rows of it:

In [ ]:
# Printing the first 3 rows of the data
display(global_solar.tail(3))

*As we already know from the About data explanation section, we will need to lowercase and rename most of the columns in the data to shorter, more simple names.*

Looking at the tails of the data, we can already notice a few column that contain NaN values, that we will need to handle in the data preprocessing stage.

Let's check the general macro overview of the data using the `info()` method and describe the data's columns using the `describe()` method:

In [ ]:
# A) Using info() to get a general overview of the data -
print('\033[1m\x1b[4mGetting a general overview of the DF\x1b[0m:', end='\n\n')
print(global_solar.info(), end='\n\n')

# B) Describing the columns:
print('-----------------------------------------------------------------------\n') # divider
# 1. Describing the DF's columns -
print('\033[1m\x1b[4mDescribing the DF\'s columns\x1b[0m:')
display(global_solar.describe())

According to the general info() overview of the data, we can notice a few issues:
- There are 15 columns containing missing values in the data.
- These columns' dtypes should be changed:
    - The `start_year` and `retired_year` should be converted to a `int64` dtype, as they represent full years.
    - The `Country`, `Region` and `Status` could be converted to the categorical `category` dtype.

The numeric columns description gives us the following information:
- Looking at the mean of the `Latitude` and `Longitude`, it seems that the majority of the solar projects are located east of the longitude equator and north of the latitude equator line, which is approximated the region between northern Africa and south-western Europe.
- There are just 5 solar projects with a fixed retired date, which are forecasted to retire from 2047 to 2058. Meanwhile, 6.3k projects have a start date that has already been reached or is forecasted to, and that looks like there are exactly 3k solar projects with no known activation dates yet in the data. Thing is, as we already know - the `start_year` column is missing for projects that are canceled or shelved, so **there are 3k solar projects that are currently canceled or shelved.**
- The average capacity of all the plants in the DF is 87 MWs, while the range vary from 10 MWs to 20k MWs with a standard deviation of about 326.7 MWs.

### 2.2 Exploring missing values

Now we will need to check the missing values in order to find possible connections between them so we could fill them later on.

I'll calculate the percentages of missing values in each column of the DF and present the missing values situation in a `missingno` plot that shows the share of non-null values in each column:

In [ ]:
# Printing a guiding note for the plot -
print('\033[1m\x1b[4mGuiding Note\x1b[0m: The \033[1m\x1b[32mgreen\x1b[0m horizontal line represents 100% full columns.\n\
              The \033[1m\x1b[33morange\x1b[0m line represents 50% full columns.\n\
              The \033[1m\x1b[31mred\x1b[0m line represent the 5% full threshold.')

# Plotting horizontal lines representing:
plt.axhline(0.5,0,1, c='orange', alpha=0.65, linewidth=1.5); # the full threshold
plt.axhline(0.05,0,1, c='red', alpha=0.65, linewidth=1.5); # half-full threshold 
plt.axhline(1,0,1, c='lightgreen', alpha=0.85, linewidth=3); # less than 5% full threshold

# Plotting the share of non-missing values in the data's columns -
msno.bar(global_solar, label_rotation=27, fontsize=18)

# Plot modifications:
plt.title('Share of non-missing values in each column', fontsize=32, pad=20, style='oblique');
plt.grid()

# Showing plot -
plt.show()

# Calculating the share of missing values in the DF -
print('\033[1m\x1b[4mPercentage of missing values from the DF (In a descending order)\x1b[0m:\n\n',\
      round(global_solar.isna().sum() / len(global_solar.isna()) * 100, ndigits=2)\
      .sort_values(ascending=False), sep='');

There are 15 columns containing missing values in the DF.

Out of these, *there are some columns that we can overlook:*
1. We already know that the "local name" columns, i.e. `Operator Name in Local Language / Script` (69.84% missing), `Owner Name in Local Language / Script` (62.89% missing) and `Project Name in Local Language / Script` (46.27% missing) are filled only when there is a local name to these available, so we can ignore these columns missing values.
2. Secondly, the `start_year` column is known to have missing values for projects that are canceled or shelved. In the same way, the `Retired year` column, which has a missing values rate of 99.95%, is only filled if the project has a destined and known retirement year.
3. The `Other Name(s)` column (82.85% missing) is only presenting alternate project names if there are any. Same goes for the `Other IDs (unit/phase)` (91.9% missing) and `Other IDs (location)` (77.01% missing) columns, which represent other IDs only if there are any.

#### *Conclusion:*

***The other columns, which will require further handling, are:***
- **`Phase Name`:** 
      contains 57.34% missing values.
- **`City`:** 
      contains 47.39% missing values.
- **`Major area (prefecture, district)`:** 
      contains 42.01% missing values.
- **`Operator`:** 
      contains 35.33% missing values.
- **`Local area (taluk, county)`:** 
      contains 34.37% missing values.
- **`Owner`:** 
      contains 9% missing values.
- **`State/Province`:** 
      contains 6.9% missing values.
      
***Note:***
Although the`Phase Name` column contains missing values, it is worthy of saying that the GEM has the `GEM phase ID` column, stating each unique project's unique phase's unique ID.

### 2.3 Searching for duplicates

In order to fish for duplicates, we will check both complete duplicates, and duplicates only by project name and countries, below:

In [ ]:
# Checking for complete duplicates -
print(f"Amount of complete duplicates: {global_solar.duplicated().sum()}")

# Checking for partial duplicates where the same projects from a country appear more than once:
# 1. Filtering data to include only necesary columns for the check -
part_dup = global_solar[['Country', 'Project Name', 'Phase Name']]
# 2. Checking for partial duplicates -
print(f"Amount of partial duplicates: {part_dup.duplicated().sum()} \
(which represents {part_dup.duplicated().sum() / len(global_solar):.3%} from the DFs rows)")

There are no complete duplicates, but it seems like there might be a few partial ones.

Let's check why the partial duplicates exist by first looking at them:

In [ ]:
# Listing the duplicates unique projects' names in a variable -
dup_proj_names = part_dup[part_dup.duplicated()]['Project Name'].tolist()

# Filtering data for rows for these specific projects -
display(global_solar[global_solar['Project Name'].isin(dup_proj_names)])

Checking the first duplicate `"Anhui Lingbi Louzhuang Photovoltaic solar farm"` project, we can see that although the two first rows seem as the same project at the same year at first sight, they represent 2 separate phases which occurred on the same year in a 2 close towns with almost similar names, that can be supported by the different GEM phase and location IDs and phase location coordinates.

Regarding the second duplicate `"Henan Yuzhou Yuke Photovoltaic Power Co., Ltd."` project, we can see much clearer than the first duplicate that these are 2 separate phases, which occurred in different locations in a gap of 2 years between them.

#### *Conclusion:*

Hence, we will leave the duplicated rows as they were.

### 2.4 Conclusions and further steps

After the initial data estimation of the raw DF, we've noticed a few issues -
- All columns' names are capitalized with spaces between words, also some of the columns names are just too long.


- These columns' dtypes should be changed:
    - The `start_year` and `retired_year` should be converted to a `int64` dtype, as they represent full years.
    - The `Country`, `Region` and `Status` could be converted to the categorical `category` dtype.
    
    
- There are 3k solar projects that are currently canceled or shelved.


- There are 15 columns containing missing values, out of which 8 columns are clearly fine to ignore. The columns who require special attention are:
    - **`Phase Name`** 
    - **`City`** 
    - **`Major area (prefecture, district)`**
    - **`Operator`** 
    - **`Local area (taluk, county)`**
    - **`Owner`** 
    - **`State/Province`** 


- While fishing for duplicates, we've found that there are 2 partially duplicated rows, that as we checked, weren't actually duplicates at all, they just seemed like it because of the NaN `Phase Name` values. Therefore the data is clear of duplicates.

The next steps will be thus:
- Modify and change the columns by lowercasing and renaming most of the columns in the data to shorter, more simple names for a cleaner analysis.
- Fixing the columns' dtypes.
- Check the consistency across the `Country` column trackers, *as the database author mentioned that although it should be ordered, it might lack grouping consistency*.
- Handling the remaining 7 missing values columns.

-------------------------

# 3. Data Preprocessing

We will use this section to fix and prepare the data to analysis.

In order to save some time, I'll import a few pre-defined functions of my own for the further work:

In [ ]:
# Defining a few helpful functions:
# -------------------------------------------------------------------------------------------------------------------------
# -------------------------------------------------------------------------------------------------------------------------
# 1. Defining a function that gets rid of the index:
def flat_Idx(df, idx_col, reset_index=True, reorder=False):
    '''
    The flat_Idx() function is used to flatten a 1-leveled index, 
    and place the original index values as a regular column before dropping the original index.
    
    Parameters:
    df: The Indexed DataFrame to flatten.
    idx_col: The name of the index column. Also the original DataFrame's Index.
    reset_index: True by default, resetting the original DataFrame's MultiIndex and dropping the old one.
    reorder: Placing the Index column/s at the beginning of the DataFrame. False by default. 
             Reordered DataFrame is returned, not inplace.
    '''
    # Salvaging the index column values -
    idx = [x for x in df.index]

    # re-Creating the originally-index column as a normal column -
    df[idx_col] = idx
    
    # Resetting index (As long as reset_index=True)-
    df.reset_index(drop=True,inplace=reset_index)

    # loop is ran only if reorder=True:
    if reorder==True:
        # Storing former-index column' name -
        col_name = df.iloc[:,-1].name
        # Storing df columns' names -
        cols = df.columns
        cols = cols.insert(0,col_name)
        return df.reindex(columns=cols).iloc[:,:-1] # The iloc removes the original salvaged index column

# -------------------------------------------------------------------------------------------------------------------------
# 2. Defining a function that insert last column as first column -
def l2f(df):
    '''
    The last_to_first() function place the last column in the beginning of the DataFrame.
    '''
    col_name = df.iloc[:,-1].name
    cols = df.columns
    cols = cols.insert(0,col_name)
    return df.reindex(columns=cols).iloc[:,:-1]

## 3.1 Fixing columns' names

Since simply lowercasing the columns won't do the whole trick, I'll replace all columns names below:

In [ ]:
# Changing the columns' names -
global_solar.columns = ['country', 'project_name', 'phase_name',
                        'project_name_local', 'other_names',
                        'capacity_mw', 'capacity_rating', 'status', 'start_year',
                        'retired_year', 'operator', 'operator_local',
                        'owner', 'owner_local', 'latitude',
                        'longitude', 'loc_accuracy', 'city', 'local_area',
                        'major_area', 'state_prov', 'region',
                        'gem_loc_id', 'gem_phase_id', 'other_loc_id',
                        'other_unit_phase_id', 'wiki_url']

# Printing the new columns' names -
print(global_solar.columns)

#### Result:

The columns names have been fixed.

##### ***Below are the new columns' names and their shortened descriptions:***
- **`country`** - Country the project is located in (Unabbreviated)


- **`project_name`** - Commonly used name.
 

- **`phase_name`** - Common name of the project phase.
 


- **`local_project_name`** - Commonly used name in the locally used script.
 


- **`other_names`** - Alternative names, including the name in local script.


- **`capacity_mw`** - Nameplate capacity, or best available data, of the phase in megawatts rounded to whole number.
 

- **`capacity_rating`** - The reported capacity rating, unknown selected if not provided.
 
 
- **`status`** - *see status definitions in the 'Introduction to the data' section at the start of the report*.


- **`start_year`** - Year the project is or is expected to be commissioned.
 
 
- **`retired_year`** - Year the plant is taken offline.
 
 
- **`operator`** - The company that operates the project.
 

- **`local_operator`** - Company name that operates the project in the locally used script.
 

- **`owner`** - The company that directly owns the plant; When owned by a special purpose vehicle, the next level of ownership is provided.


- **`local_owner`** - Company name that directly owns the project in the locally used script.
 

- **`latitude`** - Latitude, assumed WGS84 (Google maps), of the project (not by unit).


- **`longitude`** - Longitude, assumed WGS84 (Google maps), of the project (not by unit).


- **`loc_accuracy`** - Location is Approximate based on using nearest city or other information in data sources, or Location is Exact when explicitly provided in data source.


- **`city`** - City, town, village, or township (China) where project is located.


- **`local_area`** - County, taluk, or district (China) where project is located .


- **`major_area`** - Prefecture, district, or municipality (China) where project is located.


- **`state_prov`** - Subnational unit where project is located.


- **`region`** - Region location based on IRENA definition.


- **`gem_loc_id`** - GEM generated ID unique for each project.


- **`gem_phase_id`** - GEM generated ID unique for each phase.


- `other_loc_id`


- `other_unit_phase_id`


- **`wiki_url`** - URL of existing or planned GEM.wiki page.

## 3.2 Fixing data types

The data contain wrong dtypes in a few columns:
- The `start_year` and `retired_year` will be converted to a `int64` dtype, as they represent full years.
- The `country`, `region` and `status` will be converted to the categorical `category` dtype.

#### **3.2.1 The `'start_year'` and `'retired_year'` columns**

These columns will be converted to the integer dtype:

In [ ]:
# Sanity Check- Before operation:
print("Before the operation 'start_year' column's dtype :", global_solar['start_year'].dtype)
print("Before the operation 'retired_year' column's dtype :", global_solar['retired_year'].dtype)

# Changing the 'start_year' and 'retired_year' dtypes to Int64 -
global_solar['start_year'] = global_solar['start_year'].astype('Int64')
global_solar['retired_year'] = global_solar['start_year'].astype('Int64')

# Sanity Check- After operation:
print("\nAfter the operation 'start_year' column's dtype :", global_solar['start_year'].dtype)
print("After the operation 'retired_year' column's dtype :", global_solar['retired_year'].dtype)

#### *Results:*

The `'start_year'` and `'retired_year'` columns dtype has been changed from `float64` to `Int64`.

#### **3.2.2 The `country`, `region` and `status` columns**

These columns will be converted to a categorical dtype:

In [ ]:
# Sanity Check- Before operation:
print("Before the operation 'country' column's dtype :", global_solar['country'].dtype) # for 'country'
print("Before the operation 'region' column's dtype :", global_solar['region'].dtype) # for 'region'
print("Before the operation 'status' column's dtype :", global_solar['status'].dtype) # for 'status'

# Changing the columns' dtypes to category -
global_solar['country'] = pd.Categorical(global_solar.country) # for 'country'
global_solar['region'] = pd.Categorical(global_solar.region) # for 'region'
global_solar['status'] = pd.Categorical(global_solar.status) # for 'status'

# Sanity Check- After operation:
print("\nProcess being made...\n", flush=True)
print("After the operation 'country' column's dtype :", global_solar['country'].dtype) # for 'country'
print("After the operation 'region' column's dtype :", global_solar['region'].dtype) # for 'region'
print("After the operation 'status' column's dtype :", global_solar['status'].dtype) # for 'status'

#### *Results:*

The `'country'` and `'region'` `'status'` columns dtype has been changed from `object` to `category`.

## 3.3 Validating consistency across the `country` column

The database author mentioned in the 'About' section notes (added in the data introduction) that the column might lack in consistency.

As a consistent alphabetically DF will help us see a more grouped look of the data, we might like to make sure that it is sorted right. We will check it using the `unique()` function, as a consistent column would show an alphabetically ordered output with no misspelling:

In [ ]:
# Checking the unique countries are ordered alphabetically and contain no mispellings -
for c in global_solar['country'].unique():
    print(c, end=' | ')

We can notice that the `'country'` column is sorted alphabetically only partly on the first rows, but on the plus side there are no misspelling or partial duplicated country names.

To fix the problem we will sort the values alphabetically below:

In [ ]:
# Sorting the data by countries alphabetically, resetting index -
global_solar = global_solar.sort_values(by='country').reset_index(drop=True)

# Checking the new order of the data -
for c in global_solar['country'].unique():
    print(c, end=' | ')

#### Result:

The data is now organized by the consistent `country` column.

## 3.4 Handling missing values

The following columns require missing values intervention:
- **`phase_name`** - 57.34% missing values.
- **`city`** - 47.39% missing values.
- **`major_area`** - 42.01% missing values.
- **`operator`** - 35.33% missing values.
- **`local_area`** - 34.37% missing values.
- **`owner`** - 9% missing values.
- **`state_prov`** - 6.9% missing values.

### 3.4.1 Checking for connections between missing values

Since there are quite an amount of columns containing missing values, a smart step would be to begin by looking for a connection between these columns to others in order to get a lead on how to handle each one of them.

Let's begin by checking for connections between missing values using a few `missingno` plots that will plot the nothing. We'll be starting with a correlation heatmap showing the chance of missing values to appear in certain columns if other columns have missing values as well, and a dendrogram showing the appearance dependencies of missing or non-missing values:

In [ ]:
# Plotting the correlation of missing values appearance between DF's columns -
msno.heatmap(global_solar, cmap='seismic_r', label_rotation=30)

# Adding a title:
plt.title('Correlation map of missing values between columns', fontsize=32, pad=-5);

# Showing plot -
plt.show()

# Printing a divider
print('\n---------------------------------------------------------------------------------------------------------------\
---------------\n')

# Plotting a missing values dendrogram -
msno.dendrogram(global_solar, filter='bottom', p=0.99, \
                orientation='left', label_rotation=30, figsize=(14,10))

# Plot modifications:
plt.title('Co-Appearance dendrogram of missing values in the columns', fontsize=32, pad=15);
plt.yticks(fontsize=18)

# Showing plot -
plt.show()

# Printing a divider
print('\n---------------------------------------------------------------------------------------------------------------\
---------------\n')

# Printing stats:
oper_cols_miss_chance = global_solar['operator'].isna().sum() / global_solar['operator_local'].isna().sum()
oper_cols_miss_overlap = len(global_solar[(global_solar['operator'].isna()) & (global_solar['operator_local'].isna())])
operator_missing = global_solar['operator'].isna().sum()
print(f'\033[1m\x1b[4mSome stats\x1b[0m:\n\n\
Share of rows where missing values appear in both `operator` and `operator_local` columns simultanously \
is {oper_cols_miss_chance:.2%} of the rows.\n\
The amount of overlapping missing values rows between the two columns is {oper_cols_miss_overlap} rows,\n\
while he total number of missing values in the `operator` column is {operator_missing}.')

**I'll interpret the key insights from each of the plots separately below.**

#### ***Heatmap interpretation:***

*Be noted that columns which don't contain missing values were left empty. Also note that we plotted **all** columns including the ones we've decided earlier that we don't need to fill in order to see if we can fill them using other columns.*

Looking strictly at our main focus group of missing values columns, the most notable insight is that the `operator` column has a mid-leveled correlation with the `operator_local` column of 0.5, which while adding to the printed additional printed statistics, confirm **half of the rows containing missing values in the `operator_local` column are also missing all `operator` column values**, or to say is the other way - **3290 of the missing values in the `operator` column are also missing in the `operator_local` column, while only 7 of them don't share the overlap**; in 50% of the rows where one is missing - the latter is missing as well.

At a little slighter strength, the `phase_name` missing values has a light correlation of 0.3 with the `other_unit_phase_id` column that represents other IDs for than the known IDs.

A worthy notices is deserved to the local names' columns (the project, operator and owner local names) which might show strong results in the heatmap, but aren't relevant because they have a huge amount of missing values derived from the fact not every of these parameters even possess a local language name, or a non English one per-se, meaning that the null values probably overlap with the majority of the columns, so their high and distorted correlation is inadmissible to confirm any relation between columns. Also, the `start_year` and `retired_year`have a full correlation of exactly 1, because as we already know from the database author's notes, the `start_year` wasn't included for projects who got canceled or shelved so there's a high chance the same goes for the `retired_year` column because projects who got canceled or shelved can't be retired because they didn't even start, meaning ***we can say all rows who miss both the `start_year` and `retired_year` columns can be considered as shelved or canceled.***

#### ***Dendrogram interpretation:***

*Note: In order to correctly understand the dendrogram, read it from right to left. The plot is showing the appearance or disappearance correlation of connected columns, meaning that values in the columns connected by a flat line at the right are directly appearing or disappearing when the other leaf connected columns' values appear or disappear, orderly, and vice versa, while the ones connected at the most left-sided leaves have the slightest connection of correlation on appearance. In addition, the plot is filtered to include only the original 15 columns who contain missing values, disregarding the full columns.*

We can notice that the strongest connections of co-appearance between the columns are those of the `operator_local` and `owner_local`, which makes sense because we already know that most countries don't have local names except ones who aren't of roman alphabet origins, and appear or disappear mostly on the same occasions. Most of the other connections are pretty trivial.

We've already seen a few light to middle connections in between the columns, but for now it's not enough for most of the columns missing values to state of an actual connection in appearance of missing values. Therefore, in order to be certain of a lack of connection, I'll plot a matrix plot that checks the precise location of missing values in the DF, I'll keep it unordered in purpose of seeing directly if there is a connection between missing values:

In [ ]:
# Plotting the nullity matrix -
msno.matrix(global_solar, label_rotation=30)

# Plot modifications:
plt.title('Nullity matrix of the exact location of non-missing values', fontsize=32, pad=15);
plt.axhline(635,0.435,0.492, c='orange', alpha=0.25, linewidth=75);
plt.axvline(8.5,0,1, c='red', alpha=0.15, linewidth=105);

# Showing plot -
plt.show()

*Plot guide notes: The sparkline on the right shows the amount of columns containing missing values in each row, and the plot's black lines each represent non-null values for each rows at each certain column, while their absence represent the opposite.*

By now we can now clearly confirm that ***there aren't any established connections between the appearance of missing values in any of the group specific columns to the appearance of others***. 

We can see a supporting act to our previous statement while looking at all rows who miss both the `start_year` and `retired_year` values in the faded red marker on the plot, as we can clearly see both are appearing or missing in the *exact* same rows.

On the other hand, we can see both in the plot and the sparkline frequency that the missing values appear in what seems to be the same countries in a pretty similar way, that can be seen while looking at different patterns in all columns from top to bottom, as we already know the data is ordered by the countries. $For$ $example$ *look horizontally at the rows' patterns in the drawn faded orange boxes in the middle of the DF plot*.

#### **Conclusions regarding missing values correlation:**

As I stated 3 paragraphs above, except from a few connections between the columns which were left empty in purpose, there aren't any established connections between the appearance of missing values in any of the columns to the appearance of others, ***meaning we will need to handle each column individually***. Nevertheless, the strongest insight came from the matrix plot, showing that ***there might be a patterned connection between missing values in specific countries***, a fact which we can use while handling the missing values in the next steps.

*Also*, after fixing the data we could enrich the data with a new boolean column to state whether solar projects are shelved or canceled by simply checking if both `start_year` and `retired_year` are missing.

### 3.4.2 Handling the locations columns

We will use the coordinates columns `latitude` and `longitude` to fill the following columns:
- `city` (rep. by **city**) - 47.39% missing values.
- `local_area` (rep. by **county**) - 34.37% missing values.
- `major_area` (rep. by **state_district**) - 42.01% missing values.
- `state_prov` (rep. by **state**) - 6.9% missing values.

To do that action we'll use the GeoPy mapping library which converts coordinates to addresses, while we will extract each missing column's max amounts of specific values we can get. In columns where the location accuracy (`loc_accuracy`) is 'exact' and fully accurate, we'll fill all fields we could get, while in the 'approximate' accurate ones we'll fill all missing values except the `city` to keep the data integrity right.

We'll begin by creating the `'coordinates'` column that will be concatenating the `latitude` and `longitude` columns in a geographical coordinates format:

In [ ]:
# Concatenting latitude and longitude to create the 'coordinates' column -
global_solar['coordinates'] = global_solar['latitude'].map(str) + ', ' + global_solar['longitude'].map(str)

# Checking the data's values format and dtype -
print('`coordinates` column values\' format: ', global_solar['coordinates'][0])
print('`coordinates` column values\' dtype: ', type(global_solar['coordinates'][0]))

Now that the `'coordinates'` column has been created, The next step will be to create the `reverse_mapping()` function that will reverse-map each row's coordinates according to the given query parameter of each of the necessary columns and add it to an interior variable:

In [ ]:
# Defining the `reverse_geocoding()` function -
def reverse_geocoding(coordinates, q='display_name', INA='None', delay=0.00015, lang='en'):
    '''
    Function is used for geocode address mapping of latitude and longitude values;
    Simply saying - for converting coordinates to addresses.
    
    How it works:
    The function map each of the coordinates values and return the query item (i.e. `q` parameter) 
    for each coordinates pair. After that the value is appended to an exterior variable 
    that will be named `addresses`.
    
    Parameters:
    coordinates: The iterated Series or DataFrame column containing coordinates.
    q: The queried parameter; can be either one of the values available at the list below. 
       Set to 'display_name' by default.
    INA: Stands for 'item not available', 'None' by default.
       Set the returned values if `q` item is not found for a row.
    delay: Set to 0.015 seconds by default. Parameter is controlling the delay between 
       mapping requests to the API.
    lang: Set to 'en' by default. Control translation language of mapped data, that is 
       translated to English by default.
       It's possible to pass two letters country codes (ISO 639-1) to change 
       the values language.
    
    Query parameters - options:
    place_id, licence, osm_type, osm_id, lat, lon, display_name, address, suburb, city, 
    state_district, state, postcode, country, country_code, boundingbox
    '''
    # Looping over `coordinates` to map the `q` item:
    for val in coordinates:
        time.sleep(delay)
        location = geolocator.reverse(val, language=lang)
        if location != None:
            addresses.append(location.raw.get('address').get(q))
        else:
            addresses.append(INA)
                
# Creating a Geolocator mapping object -
geolocator = Nominatim(user_agent="test")

Now that the preparations are set, we can start mapping all missing values.

The process will be done for each column's missing values separately.

<div style="border:solid black 2px; padding: 10px"> <h1 style="color:red; margin-bottom:20px">A strong notice!</h1>
    
    
<p style="font-size: 16px">Since the map API takes time to process each request and cannot receive too many parsing requests simultanously, <strong>the mapping process will take some time to map all data with the added delay between each request (Averagely 75:16 minutes).</strong></p>

#### $1.$ Mapping and filling `state/prov`:

First we'll create a filtered DF containing all rows where the column has a missing values, and then we will begin mapping the state / province column values, ***process is expected to take around 5-6 minutes (5:23 mins average)***:

In [ ]:
# A) Filtering data to rows where the location accuracy is set to 'exact', that contain `state_prov` missing values -
locs_state = global_solar[(global_solar['state_prov'].isna())]

# Printing the amount of missing values' rows to iterate over the function -
print(f'Amount of missing rows: {len(locs_state)}', end='\n\n')

# ------------------------------------------------------------------------------------------------------------------------
# B) Mapping states from `coordinates`:
# 1. Creating a blank 'addresses' variable -
addresses = []

## Counting action time
start = perf_counter()
print(f'Starting timer at: {start}')

# 2. Mapping all missing `state_prov` values translated to English -
reverse_geocoding(locs_state['coordinates'], q='state')

## Printing total action time
end = perf_counter()
print(f'Stopping timer at: {end}\n\
Execution time - {end - start}')

# 3. Saving the addresses to a unique var -
missing_states = addresses

Now that we have all missing values in the `missing_states` variable, we'll use it to fill all missing values in the `state_prov` column by simply overwriting it, since the filtered DF had originally only null values in it:

In [ ]:
# Sanity check BEFORE action -
print(f"Amount of missing values before action: {locs_state['state_prov'].isna().sum()}")

# Filling missing values in the `state_prov` column
# by pasting the `missing_states` mappeing list variable on it -
locs_state['state_prov'] = missing_states

# Sanity check AFTER action -
print(f"Amount of missing values after action: {locs_state['state_prov'].isna().sum()}")


We mapped 252 locations out of the 647 missing values in the `'state_prov'` column. Not all countries are divided to states or provinces so it should be satisfying enough.

Now we need to fill the original `global_solar` DF with the values we've filled the `locs_state` interim DF.

**To start with it, we'll do the following:**
1. In this stage we'll:
    - Convert the `locs_state` filtered DF and filter it to include only the `'state_prov'` column. We'll flatten the index to a column while keeping the old index.
    - Create another filtered DF of all non-missing `'state_prov'` column rows from the original `global_solar` DF and flatten the index to a column while keeping the original index without any resettings.
2. We'll use both `'index'` columns as merging keys to join both DFs to one series, and then we'll overwrite the original column with the merged `state_prov` column of the merged filtered DFs.

I'll define the following `df2s2df()` function to automate the process for the rest of the process and for the next columns filling:

In [ ]:
# Defining the `df2s2df()` function to help with automation -
def df2s2df(df, col):
    '''
    Function automate the process of preparing DataFrames to missing values filling.
    
    Parameters:
    df: Iterated DataFrame.
    col: The column that is destined to be filled in later process.
    '''
    # A) Taking care of the `df` data pre-merger:
    # 1. Converting the `col` column from the `df` to a Series -
    df = df[col]

    # 2. Converting the Series to a DataFrame -
    ret_df = pd.DataFrame(data=df)

    # 3. Flattening index to a column -
    flat_Idx(ret_df,'index',reset_index=False)

    # 4. Using l2f() to insert index column as first column -
    ret_df = l2f(ret_df)
    return ret_df

We'll begin ***stage one*** below:

In [ ]:
# A) Taking care of the `locs_state` data pre-merger:
# 1. Conducting automation process using the df2s2df() function -
locs_state = df2s2df(locs_state,'state_prov')

# 2. Check DF's new look -
print(f'Amount of rows in the missing values DF: {len(locs_state)}')
display(locs_state.head(3))

# ------------------------------------------------------------------------------------------------------------------------
# B) Create the `non_miss_rows` DF pre-merger,
#    DF will contain all non-missing values of the `state_prov` column and an 'index' column:
# 1. Making a filtered DF of all non-missing values on the `state_prov` column on the original DF -
non_miss_rows = global_solar[~(global_solar['state_prov'].isna())]

# 2. Conducting automation process using the df2s2df() function -
non_miss_rows = df2s2df(non_miss_rows, 'state_prov')

# 3. Check DF's new look -
print(f'\nAmount of rows in the non-missing values DF: {len(non_miss_rows)}')
display(non_miss_rows.head(3))

The 2 filtered DFs were created. Now we need to merge them using the `append()` function and overwrite the original column in the basic DF:

In [ ]:
# A) Appending both filt DFs so that they'll merge to cover all original DF's rows:
# 1. Appending both filt DFs together -
state_col = locs_state.append(non_miss_rows)

# 2. Sorting data by the 'index' column -
state_col.sort_values('index', inplace=True)

# -------------------------------------------------------------------------------------------------------------------------
# B) Filling missing values by 
#    Overwriting the `state_prov` column by the merged filt DF's correlating column:
# Sanity check BEFORE action -
print(f"Amount of missing values before action: {global_solar['state_prov'].isna().sum()}")

# Filling the mapped states into the original column by overwriting it -
global_solar['state_prov'] = state_col['state_prov']

# Sanity check AFTER action -
print(f"Amount of missing values after action: {global_solar['state_prov'].isna().sum()}")

The reverse mapping process mapped 252 locations out of the 647 missing values in the `state_prov` column, and the mapped locations were filled into the DF.

Now we will fill the rest of the missing values with the value 'unavailable', because either the countries aren't divided to states or the data just couldn't be mapped:

In [ ]:
# Sanity check BEFORE action -
print(f"Amount of missing values before action: {global_solar['state_prov'].isna().sum()}")

# Filling remaining missing values with 'unknown' -
global_solar['state_prov'] = global_solar['state_prov'].fillna('Unavailable')

# Sanity check AFTER action -
print(f"Amount of missing values after action: {global_solar['state_prov'].isna().sum()}")

*Result:*

We were able to map and fill 252 locations out of the 647 missing values in the `state_prov` column. The rest of the missing values were filled with the value `'Unavailable'`. Column is completely filled

#### $2.$ Mapping and filling `major_area`:

First we'll create a filtered DF containing all rows where the column has a missing values, and then we will begin mapping the district column values, ***process is expected to take around 26-28 minutes (26:50 mins average):***:

In [ ]:
# A) Filtering data to rows that contain `major_area` missing values -
locs_dist = global_solar[(global_solar['major_area'].isna())]
print(f'Amount of missing rows: {len(locs_dist)}', end='\n\n')

# ------------------------------------------------------------------------------------------------------------------------
# B) Mapping districts from `coordinates`:
# 1. Creating a blank 'addresses' variable -
addresses = []

## Counting action time
start = perf_counter()
print(f'Starting timer at: {start}')

# 2. Mapping all missing `major_area` values translated to English -
reverse_geocoding(locs_dist['coordinates'], q='state_district', delay=0.00000000000000000000000015)

## Printing total action time
end = perf_counter()
print(f'Stopping timer at: {end}\n\
Execution time - {end - start}')

# 3. Saving the addresses to a unique var -
missing_dist = addresses

Now that we have all missing values in the `missing_states` variable, we'll use it to fill all missing values in the `major_area` column by simply overwriting it, since the filtered DF had originally only null values in it:

In [ ]:
# Sanity check BEFORE action -
print(f"Amount of missing values before action: {locs_dist['major_area'].isna().sum()}")

# Filling missing values in the `major_area` column
# by pasting the `missing_dist` mapping list variable on it -
locs_dist['major_area'] = missing_dist

# Sanity check AFTER action -
print(f"Amount of missing values after action: {locs_dist['major_area'].isna().sum()}")


We mapped 582 locations out of the 3920 missing values in the `'major_area'` column.

Now we need to fill the original `global_solar` DF with the values we've filled the `locs_dist` interim DF.

**To start with it, we'll conduct the same process that was done with last columns - use the `df2s2df()` function to automate the process, append the filtered DFs and overwrite the missing values column.***

In [ ]:
# A) Taking care of the `locs_dist` data pre-merger:
# Conducting automation process using the df2s2df() function -
locs_dist = df2s2df(locs_dist,'major_area')

# ------------------------------------------------------------------------------------------------------------------------
# B) Create the `non_miss_rows` DF pre-merger,
#    DF will contain all non-missing values of the `major_area` column and an 'index' column:
# 1. Making a filtered DF of all non-missing values on the `major_area` column on the original DF -
non_miss_rows = global_solar[~(global_solar['major_area'].isna())]

# 2. Conducting automation process using the df2s2df() function -
non_miss_rows = df2s2df(non_miss_rows, 'major_area')

# ------------------------------------------------------------------------------------------------------------------------
# C) Appending both filt DFs so that they'll merge to cover all original DF's rows:
# 1. Appending both filt DFs together -
dist_col = locs_dist.append(non_miss_rows)

# 2. Sorting data by the 'index' column -
dist_col.sort_values('index', inplace=True)

# -------------------------------------------------------------------------------------------------------------------------
# D) Filling missing values by 
#    Overwriting the `major_area` column by the merged filt DF's correlating column:
# Sanity check BEFORE action -
print(f"Amount of missing values before action: {global_solar['major_area'].isna().sum()}")

# Filling the mapped states into the original column by overwriting it -
global_solar['major_area'] = dist_col['major_area']

# Sanity check AFTER action -
print(f"Amount of missing values after action: {global_solar['major_area'].isna().sum()}")

Now we will fill the rest of the missing values with the value 'unavailable', because the data just couldn't be mapped:

In [ ]:
# Sanity check BEFORE action -
print(f"Amount of missing values before action: {global_solar['major_area'].isna().sum()}")

# Filling remaining missing values with 'unknown' -
global_solar['major_area'] = global_solar['major_area'].fillna('Unavailable')

# Sanity check AFTER action -
print(f"Amount of missing values after action: {global_solar['major_area'].isna().sum()}")

*Result:*

We were able to map and fill 582 locations out of the 3920 missing values in the `major_area` column. The rest of the missing values were filled with the value `'Unavailable'`. Column is filled.

#### $3.$ Mapping and filling `local_area`:

First we'll create a filtered DF containing all rows where the column has a missing values, and then we will begin mapping the county column values, ***process is expected to take around 26-27 minutes (26:43 mins average)***:

In [ ]:
# A) # Filtering data to rows that contain `local_area` missing values -
locs_local = global_solar[(global_solar['local_area'].isna())]
print(f'Amount of missing rows: {len(locs_local)}', end='\n\n')

# ------------------------------------------------------------------------------------------------------------------------
# B) Mapping districts from `coordinates`:
# 1. Creating a blank 'addresses' variable -
addresses = []

## Counting action time
start = perf_counter()
print(f'Starting timer at: {start}')

# 2. Mapping all missing `local_area` values translated to English -
reverse_geocoding(locs_local['coordinates'], q='county', delay=0.000000000015)

## Printing total action time
end = perf_counter()
print(f'Stopping timer at: {end}\n\
Execution time - {end - start}')

# 3. Saving the addresses to a unique var -
missing_county = addresses

Now that we have all missing values in the `missing_county` variable, we'll use it to fill all missing values in the `local_area` column by simply overwriting it, since the filtered DF had originally only null values in it:

In [ ]:
# Sanity check BEFORE action -
print(f"Amount of missing values before action: {locs_local['local_area'].isna().sum()}")

# Filling missing values in the `local_area` column
# by pasting the `missing_county` mapping list variable on it -
locs_local['local_area'] = missing_county

# Sanity check AFTER action -
print(f"Amount of missing values after action: {locs_local['local_area'].isna().sum()}")


We mapped 1309 locations out of the 3207 missing values in the `'local_area'` column.

Now we need to fill the original `global_solar` DF with the values we've filled the `locs_local` interim DF.

**To start with it, we'll conduct the same process that was done with last columns - use the `df2s2df()` function to automate the process, append the filtered DFs and overwrite the missing values column.***

In [ ]:
# A) Taking care of the `locs_local` data pre-merger:
# Conducting automation process using the df2s2df() function -
locs_local = df2s2df(locs_local,'local_area')

# ------------------------------------------------------------------------------------------------------------------------
# B) Create the `non_miss_rows` DF pre-merger,
#    DF will contain all non-missing values of the `local_area` column and an 'index' column:
# 1. Making a filtered DF of all non-missing values on the `local_area` column on the original DF -
non_miss_rows = global_solar[~(global_solar['local_area'].isna())]

# 2. Conducting automation process using the df2s2df() function -
non_miss_rows = df2s2df(non_miss_rows, 'local_area')

# ------------------------------------------------------------------------------------------------------------------------
# C) Appending both filt DFs so that they'll merge to cover all original DF's rows:
# 1. Appending both filt DFs together -
local_col = locs_local.append(non_miss_rows)

# 2. Sorting data by the 'index' column -
local_col.sort_values('index', inplace=True)

# -------------------------------------------------------------------------------------------------------------------------
# D) Filling missing values by 
#    Overwriting the `local_area` column by the merged filt DF's correlating column:
# Sanity check BEFORE action -
print(f"Amount of missing values before action: {global_solar['local_area'].isna().sum()}")

# Filling the mapped states into the original column by overwriting it -
global_solar['local_area'] = local_col['local_area']

# Sanity check AFTER action -
print(f"Amount of missing values after action: {global_solar['local_area'].isna().sum()}")

Now we will fill the rest of the missing values with the value 'unavailable', because the data just couldn't be mapped:

In [ ]:
# Sanity check BEFORE action -
print(f"Amount of missing values before action: {global_solar['local_area'].isna().sum()}")

# Filling remaining missing values with 'unknown' -
global_solar['local_area'] = global_solar['local_area'].fillna('Unavailable')

# Sanity check AFTER action -
print(f"Amount of missing values after action: {global_solar['local_area'].isna().sum()}")

*Result:*

We were able to map and fill 1309 locations out of the 3207 missing values in the `local_area` column. The remaining 1898 values were filled with the value `'Unavailable'`.

#### $4.$ Mapping and filling accurate `city`:

For `city`, we will only fill the values for rows with an `exact` location accuracy coordinates to keep data integrity. **The rows where location accuracy is 'approximate' will be left blank**.

First we'll create a filtered DF containing all rows where the column has a missing values, and then we will begin mapping the city column values, ***process is expected to take around 16-17 minutes (16:20 mins average)***:

In [ ]:
# A) Filtering data to rows where the location accuracy is set to 'exact', that contain `city` missing values -
exact_locs_city = global_solar[(global_solar['city'].isna()) & (global_solar['loc_accuracy']=='exact')]
print(f'Amount of missing rows: {len(exact_locs_city)}', end='\n\n')

# ------------------------------------------------------------------------------------------------------------------------
# B) Mapping cities from `coordinates`:
# 1. Creating a blank 'addresses' variable -
addresses = []

## Counting action time
start = perf_counter()
print(f'Starting timer at: {start}')

# 2. Mapping all accurate missing `city` values translated to English -
reverse_geocoding(exact_locs_city['coordinates'], q='county')

## Printing total action time
end = perf_counter()
print(f'Stopping timer at: {end}\n\
Execution time - {end - start}')

# 3. Saving the addresses to a unique var -
missing_city = addresses

Now that we have all missing values in the `missing_city` variable, we'll use it to fill all missing values in the `city` column by simply overwriting it, since the filtered DF had originally only null values in it:

In [ ]:
# Sanity check BEFORE action -
print(f"Amount of missing values before action: {exact_locs_city['city'].isna().sum()}")

# Filling missing values in the `city` column
# by pasting the `missing_city` mapping list variable on it -
exact_locs_city['city'] = missing_city

# Sanity check AFTER action -
print(f"Amount of missing values after action: {exact_locs_city['city'].isna().sum()}")


We mapped 1618 locations out of the 1964 missing values in the `'city'` column.

Now we need to fill the original `global_solar` DF with the values we've filled the `exact_locs_city` interim DF.

**To start with it, we'll conduct the same process that was done with last columns - use the `df2s2df()` function to automate the process, append the filtered DFs and overwrite the missing values column.***

In [ ]:
# A) Taking care of the `exact_locs_city` data pre-merger:
# Conducting automation process using the df2s2df() function -
exact_locs_city = df2s2df(exact_locs_city,'city')

# ------------------------------------------------------------------------------------------------------------------------
# B) Create the `non_miss_rows` DF pre-merger,
#    DF will contain all non-missing values of the `city` column and an 'index' column:
# 1. Making a filtered DF of all non-missing values on the `city` column on the original DF -
non_miss_rows = global_solar[~(global_solar['city'].isna()) | \
                             ((global_solar['city'].isna()) & (global_solar['loc_accuracy']=='approximate'))]

# 2. Conducting automation process using the df2s2df() function -
non_miss_rows = df2s2df(non_miss_rows, 'city')

# ------------------------------------------------------------------------------------------------------------------------
# C) Appending both filt DFs so that they'll merge to cover all original DF's rows:
# 1. Appending both filt DFs together -
city_col = exact_locs_city.append(non_miss_rows)

# 2. Sorting data by the 'index' column -
city_col.sort_values('index', inplace=True)

# -------------------------------------------------------------------------------------------------------------------------
# D) Filling missing values by 
#    Overwriting the `city` column by the merged filt DF's correlating column:
# Sanity check BEFORE action -
print(f"Amount of missing values before action: {global_solar['city'].isna().sum()}")

# Filling the mapped states into the original column by overwriting it -
global_solar['city'] = city_col['city']

# Sanity check AFTER action -
print(f"Amount of missing values after action: {global_solar['city'].isna().sum()}")

Now we will fill the rest of the missing values with the value 'unavailable', because the data just couldn't be mapped:

In [ ]:
# Sanity check BEFORE action -
print(f"Amount of missing values before action: {global_solar['city'].isna().sum()}")

# Filling remaining missing values with 'unknown' -
global_solar['city'] = global_solar['city'].fillna('Unavailable')

# Sanity check AFTER action -
print(f"Amount of missing values after action: {global_solar['city'].isna().sum()}")

*Result:*

We were able to map and fill 1618 exactly accurate locations out of the 1964 missing values with an `'exact'` location accuracy, and from a total amount of 4422 missing values in the `city` column itself. The rest of the missing values were filled with the value `'Unavailable'`.

### Handling `operator` (35.33% missing values)

Since we already have the local language `operator` names at our disposal, we'll create a variable containing a query where `operator` is missing and `operator_local` is not.

We'll translate the rows where the local name wasn't missing to fill the operator's name for each project row available, **while the intranslatable rows will be left as null values:**

In [ ]:
# A) Creating counter for rows where 
# the `operator` name is available only on the local language (`operator_local`) -
local_op = global_solar[(global_solar['operator'].isna()) & (global_solar['operator_local'].notna())]

# -------------------------------------------------------------------------------------------------------------------------
# B) Filling missing values where operator local name isn't missing and operator English name does:
# Sanity check BEFORE action -
print(f"Amount of missing values before action: {global_solar['operator'].isna().sum()}")

# Printing amount of queried missing values that could be translated - 
print(f'The query has found that {len(local_op)} rows are fitting for translation')
# Translating and filling available values -
for idx in local_op.index:
    local_name = local_op.loc[idx, 'operator_local']
    global_solar.loc[idx, 'operator'] = ts.google(local_name)
    print('\n.')

# Sanity check AFTER action -
print(f"Amount of missing values after action: {global_solar['operator'].isna().sum()}")

Resulting from the process, 7 rows were translated from the local name column. The rest of the values will be filled with 'Unknown':

In [ ]:
# Sanity check BEFORE action -
print(f"Amount of missing values before action: {global_solar['operator'].isna().sum()}")

# Filling remaining missing values with 'unknown' -
global_solar['operator'] = global_solar['operator'].fillna('Unknown')

# Sanity check AFTER action -
print(f"Amount of missing values after action: {global_solar['operator'].isna().sum()}")

*Result:*

We've managed to translate 7 rows from the `operator_local` foreign language/s and fill their missing `operator` column values. The other 3290 rows were filled as `'Unknown'`.

### Handling `owner` (9% missing values)

Same goes for the `owner` column, which contains a local name column as well. We'll translate the rows where the local name wasn't missing to fill the owner's name for each project row available:

In [ ]:
# Creating counter for rows where 
# the `owner` name is available only on the local language (`owner_local`) -
local_owner = global_solar[(global_solar['owner'].isna()) & (global_solar['owner_local'].notna())]

# -------------------------------------------------------------------------------------------------------------------------
# B) Filling missing values where owner local name isn't missing and owner English name does:
# Sanity check BEFORE action -
print(f"Amount of missing values before action: {global_solar['owner'].isna().sum()}")

# Printing amount of queried missing values that could be translated -
print(f'The query has found that {len(local_owner)} rows are fitting for translation')
# Translating and filling available values -
for idx in local_owner.index:
    local_name = local_owner.loc[idx, 'owner_local']
    global_solar.loc[idx, 'owner'] = ts.google(local_name)
    print('\n.')

# Sanity check AFTER action -
print(f"Amount of missing values after action: {global_solar['owner'].isna().sum()}")

Resulting from the process, one row was translated from the local name column. The rest of the values will be filled with 'Unknown':

In [ ]:
# Sanity check BEFORE action -
print(f"Amount of missing values before action: {global_solar['owner'].isna().sum()}")

# Filling remaining missing values with 'unknown' -
global_solar['owner'] = global_solar['owner'].fillna('Unknown')

# Sanity check AFTER action -
print(f"Amount of missing values after action: {global_solar['owner'].isna().sum()}")

*Result:*

We were only able to fill one single row of `owner` using the local name column. The other 839 were filled s `'Unknown'`.

### Handling `phase_name` (57.34% missing values)

There was no accurate data neither on the raw xlsx database file nor the GEM website whether the column relates to a progressive phase, i.e. a stage of development, or to electrical phases, as their data sources regarding our data are pretty vague about that column.

Nevertheless, the most doable option of filling the `phase_name` column is to use the GEM phase IDs (i.e. `gem_phase_id`), which although it's nice having them on the data, don't have any records on their sources to parse from, so that column is useless for us as well.

The only hope we have for filling a part of the column is the `other_unit_phase_id` column, which include other phases or units IDs, so our plan will be thus:
1. Since the filler column contains both phases and units IDs, we'll need to check the unique "other IDs" to know what values to ignore (e.g. ones that represent units, which aren't relevant). We'll compare them to the unique `phase_name` we already have in order to see what fits to be filled.
2. If there are any suitable, we'll fill the `phase_name` with them. ***The remaining of the column will be left as it is to protect data integrity.***

We'll begin by observing the unique values of the `phase_name` and the `other_unit_phase_id` columns to find a common base:

In [ ]:
# Variable of all unique `phase_name` values -
phase_name_uniq = global_solar[(~global_solar['phase_name'].isna())]['phase_name'].unique()

# Print unique values where not missing -
print(f'\033[1m\x1b[4mUnique values of the `phase_name` column\x1b[0m:')
display(phase_name_uniq)

# Variable of all unique `other_unit_phase_id` values where `phase_name` is missing -
phases = global_solar[(global_solar['phase_name'].isna()) & \
                      global_solar['other_unit_phase_id'].notna()]['other_unit_phase_id'].unique()

# Print unique values where missing -
print(f'\033[1m\x1b[4mUnique values of the `other_unit_phase_id` column\x1b[0m:')
display(phases)

We can notice that most of the IDs are meaningless, but the only-numeric ones do fit the profile and look similar to the `phase_name` approximate format. The other IDs look too encrypted to be anything but an encrypted ID, and a few rows just contain notes about overlapping area with other solar farms.

Hence, the only option that we could think about out of those is filling missing values only for rows with numeric-only `other_unit_phase_id`, we'll start by scraping only the numeric string from the unique values list:

In [ ]:
# Defining a function that 
# could extract the only-numeric items from the unique values list -
def numeric_app(list, counter):
    '''
    The `numeric_app()` function will append only numeric values from a string type list or Series to the counter.
    
    Parameters:
    list: The iterated list, Series.
    counter: An empty counter variable outside the function.
    '''
    for i in list:
        if str(i).isdigit() == True:
            counter.append(i)
        else:
            continue

# Creating a counter for the missing values rows -
filled_IDs = []

# Finding rows where the 'other ID' contains only numeric characters -
numeric_app(phases, filled_IDs)

Now that we've pulled all numeric values from the column, we'll use the same methods we used while filling the null location columns' values to fill the original DF's `phase_name` values that we've managed to find:

In [ ]:
# A) Create the `filling_var` DF pre-merger, DF will contain all missing values of the `phase_name` column
#    that have an 'other ID' value that is in the `filled_IDs` variable, and an 'index' column:
# 1. Finding rows where `phase_name` is null
#    and the other ID is in the `filled_IDs` variable -
filling_var = global_solar[(global_solar['phase_name'].isna()) & \
                           global_solar['other_unit_phase_id'].isin(filled_IDs)]

# 2. Automated 'DF >> Series >> DF' process -
filling_var = df2s2df(filling_var, 'other_unit_phase_id')

# 3. Changing columns' names -
filling_var.columns = ['index', 'phase_name']

# -------------------------------------------------------------------------------------------------------------------------
# B) Create the `non_miss_rows` DF pre-merger,
#    DF will contain all non-missing values of the `phase_name` column and an 'index' column:
# 1. Making a filtered DF of all non-missing values on the `phase_name` column on the original DF -
non_miss_rows = global_solar[~(global_solar['phase_name'].isna())]

# 2. Conducting automation process using the df2s2df() function -
non_miss_rows = df2s2df(non_miss_rows, 'phase_name')

# ------------------------------------------------------------------------------------------------------------------------
# C) Appending both filt DFs so that they'll merge to cover all original DF's rows:
# 1. Appending both filt DFs together -
phase_col = filling_var.append(non_miss_rows)

# 2. Sorting data by the 'index' column -
phase_col.sort_values('index', inplace=True)

# -------------------------------------------------------------------------------------------------------------------------
# D) Filling missing values by 
#    Overwriting the `phase_name` column by the merged filt DF's correlating column:
# Sanity check BEFORE action -
print(f"Amount of missing values before action: {global_solar['phase_name'].isna().sum()}")

# Filling the mapped states into the original column by overwriting it -
global_solar['phase_name'] = phase_col['phase_name']

# Sanity check AFTER action -
print(f"Amount of missing values after action: {global_solar['phase_name'].isna().sum()}")

*Result:*

We've managed to fill 28 rows out of the 5350 missing values in the `phase_name` column using the non-encrypted other IDs in the `other_unit_phase_id` column. Since missing values take too much big of a share from the column, we will leave the rest of the values as they were.

### 3.4.3 Final Missing Values Check

Let's check where are we standing now regarding missing values amounts, knowing that we have still left some missing values in some  columns:

In [ ]:
# Printing a guiding note for the plot -
print('\033[1m\x1b[4mGuiding Note\x1b[0m: The \033[1m\x1b[32mgreen\x1b[0m horizontal line represents 100% full columns.\n\
              The \033[1m\x1b[33morange\x1b[0m line represents 50% full columns.\n\
              The \033[1m\x1b[31mred\x1b[0m line represent the 5% full threshold.')

# Plotting horizontal lines representing:
plt.axhline(0.5,0,1, c='orange', alpha=0.65, linewidth=1.5); # the full threshold
plt.axhline(0.05,0,1, c='red', alpha=0.65, linewidth=1.5); # half-full threshold 
plt.axhline(1,0,1, c='lightgreen', alpha=0.85, linewidth=3); # less than 5% full threshold

# Plotting the same missing values plot we plotted at the start of the section -
msno.bar(global_solar, label_rotation=27, fontsize=18)

# Plot modifications:
plt.title('Share of non-missing values in each column', fontsize=32, pad=20, style='oblique');
plt.grid()

# Showing plot -
plt.show()

# Calculating the share of missing values in the DF -
print('\033[1m\x1b[4mPercentage of missing values from the DF (In a descending order)\x1b[0m:\n\n',\
      round(global_solar.isna().sum() / len(global_solar.isna()) * 100, ndigits=2)\
      .sort_values(ascending=False), sep='');

### ***3.4.4 Conclusions regarding missing values:***

Out of the 7 columns we've decided worth handling, we've filled the group of columns thus:
1. we've mapped the following location columns coordinates to finds the following addresses:**
    - **`state_prov`** - From 6.9% to 4.23% missing values.
        - ***252 values were filled out of 647***

    - **`major_area`** - From 42.01% to 35.77% missing values.
        - ***582 values were filled out of 3920***

    - **`local_area`** - From 34.37% to 20.34% missing values.
        - ***1309 values were filled out of 3207***

    - **`city`** - From 47.39% to 30.05% missing values.
        - ***1618 values were filled out of 4422***
- ***The rest of the missing values in the above columns were filled with the value `'Unavailable'`.***

2. We've used the numeric and non encrypted other IDs to fill `phase_name`:**
    - **`phase_name`** - From 57.34% to 57.04% missing values.
        - ***28 values were filled out of 5350***


3. For missing values with local names on these columns, we've translated local names to fill values:**
    - **`operator`** - From 35.33% to 35.26% missing values.
        - ***7 values were filled out of 3290***
    - **`owner`** - From 9% to 8.99% missing values.
        - ***1 values were filled out of 840***
- ***The rest of the missing values in the above columns were filled with the value `'Unknown'`.***

## 3.5 Checking fixed data

Let's have a look at the patched up DF:

In [ ]:
# A) Using info() to get a general overview of the data -
print('\033[1m\x1b[4mGetting a general overview of the DF\x1b[0m:', end='\n\n')
print(global_solar.info(), end='\n\n')

# B) Describing the columns:
print('-----------------------------------------------------------------------\n') # divider
# 1. Describing the DF's columns -
print('\033[1m\x1b[4mDescribing the DF\'s columns\x1b[0m:')
display(global_solar.describe())

# C) Sampling a random 2 rows from the DF -
print('-----------------------------------------------------------------------\n') # divider
print('\033[1m\x1b[4mSampling 2 random rows from the DF\x1b[0m:', end='\n\n')
display(global_solar.sample(2))

*Conclusion:*

Although there are still some faults in the data, with our current data sources we've patched it the most we could.

## 3.6 Conclusions regarding Data Preprocessing

In this data preprocessing chapter:
1. **We've fixed the columns' names** to lowercased names formatted in a 'snake_case' format.
2. **We've fixed wrong columns' data types:**
    - The `'start_year'` and `'retired_year'` columns dtype has been changed from `float64` to `Int64`.
    - The `'country'` and `'region'` `'status'` columns dtype has been changed from `object` to `category`.
3. **We've validated the `'country'` column data**, sorting the data by country name alphabetically. We have also checked for duplicated countries and misspellings and found none. 

**4. *Handled the missing values***

To decide how should we handle missing values, **we've first explored all missing values for possible connections** using some missingno missing values plots, **and we've found that overall there aren't any fully based connections between missing values**, except:
- The `'start_year'` and `'retired_year'` columns had a full correlation of 1, which makes sense because the database author has stated that the starting year for projects who got canceled or shelved was left missing, so projects who got canceled cannot be retired.
- Using the matrix plot, we've seen that missing values tend to look differently in different rows groups locations on the table, hinting there might be some connection between the missing values while looking at the columns for rows of similar countries. We've also seen that some missing rows of the the owner and operator columns seems to have full values on their correlating local name columns.

After finding there isn't any based connection between the columns' missing values, We started dealing with the missing columns individually:
1. I've decided to fill the missing values in the location column with their coordinates values. We've created the `'coordinates'` column, that created a string dtyped column containing the latitude and longitude in the accepted format of it. After that I've created a reversed mapping function that is used for mapping the coordinates exact addresses. We used that function to map the missing values in the following columns, while the rest of the missing values were filled with the value 'Unavailable':
    - We mapped and filled 252 locations out of the 647 missing values in the `state_prov` column.
    - We mapped and filled 582 locations out of the 3920 missing values in the `major_area` column.
    - We mapped and filled 1309 locations out of the 3207 missing values in the `local_area` column.
    - We mapped and filled 1618 exactly accurate locations out of the 1964 missing values with an `'exact'` location accuracy, and from a total amount of 4422 missing values in the `city` column itself (That's because mapping the city of approx. accurate coordinates will not be prolific).


2. For missing values with local names on the 'local name' columns, we've translated local names to fill values, filling the remaining missing values with the value 'Unknown:
    - **`operator`** - 7 values were filled out of 3290.
    - **`owner`** - One value was filled out of 840.


3. We've managed to fill 28 rows out of the 5350 missing values in the `phase_name` column using the non-encrypted other IDs in the `other_unit_phase_id` column. Since missing values take too much big of a share from the column, we left the rest of the values as they were.








--------------------

# 4. The Analysis

Now that the data is ready, in the following section of the report, we'll visualize and analyze the data to deeper tenses, as we'll explore:
- The energy output of solar farms. We will also map the solar farms by capacity throughout the world.
- The total global growth in the solar industry.
- The people who possess the biggest shares of the global solar energy market.

I'll add another 2 functions that could ease the further analysis:

In [ ]:
# Defining a display funtion that allows to display multiple DFs side by side -
def display_sided(*args,titles=cycle([''])):
    html_str=''
    for df,title in zip(args, chain(titles,cycle(['</br>'])) ):
        html_str+='<th style="text-align:center"><td style="vertical-align:top">'
        html_str+=f'<h3 style="text-align: center;">{title}</h2>'
        html_str+=df.to_html().replace('table','table style="display:inline"')
        html_str+='</td></th>'
    display_html(html_str,raw=True)
    
# Defining a function that gets rid of the double index:
def flat_TripleIdx(df, idx1_col, idx2_col, idx3_col=None, reset_index=True, reorder=False, dub=False):
    '''
    The flat_TripleIdx() function is used to flatten 3-levels MultiIndex (or Double-Index), 
    and split the tupled indices to separate columns before dropping the original index.
    
    Parameters:
    df: The MultiIndexed DataFrame to flatten.
    idx1_col: The name of the index tuple first column. Also the 1st-level in the original DataFrame's MultiIndex.
    idx2_col: The name of the index tuple second column. Also the 2nd-level in the original DataFrame's MultiIndex.
    idx3_col: The name of the index tuple third column. Also the 3nd-level in the original DataFrame's MultiIndex.
    reset_index: True by default, resetting the original DataFrame's MultiIndex and dropping the old one.
    reorder: Placing the Index column/s at the beginning of the DataFrame. False by default. 
             Currently designed only to flatten a 2-leveled MultiIndex.
    **kwargs
    '''
    
    # Flattening the `df` double (2-leveled) index -
    df.index = df.index.to_flat_index()

    # Salvaging the indices values from the tupled indices -
    idx_1 = [x[0] for x in df.index]
    idx_2 = [x[1] for x in df.index]
    if dub==False:
        idx_3 = [x[2] for x in df.index]

    # re-Creating the `idx_1`, `idx_2` and `idx_3` columns -
    df[idx1_col] = idx_1
    df[idx2_col] = idx_2
    if dub==False:
        df[idx3_col] = idx_3
        
    # Resetting index (As long as reset_index=True)-
    df.reset_index(drop=True,inplace=reset_index)
    
    # Placing the former index tuple's columns at the start of the DataFrame -
    if reorder==True:
        # Storing former-index column' name -
        col_name_fir = df[idx1_col].name
        col_name_sec = df[idx2_col].name
        if dub==False:
            col_name_thi = df[idx3_col].name
        
        # Reordering the columns -
        # Storing columns in interior variables -
        fir_col = df.pop(col_name_fir)
        sec_col = df.pop(col_name_sec)
        if dub==False:
            thi_col = df.pop(col_name_thi)
        # Inserting the columns at the beginning of the DataFrame -
        if dub==False:
            df.insert(0,col_name_thi,thi_col)
        df.insert(0,col_name_sec,sec_col)
        df.insert(0,col_name_fir,fir_col)

# -------------------------------------------------------------------------------------------------------------------------
# Defining the top_n() function, which creates leaderboards for columns -
def top_n(df, lboard_col, top5_parameter, top=5, ascend=False, xtra_lb_col=True, ranked_idx=True):
    '''
    The top_n() function takes a DataFrame\'s column (i.e. leaderboard column), sort it,
    and return the top n rows after it is sorted.
    
    Parameters:
    df: DataFrame.
    lboard_col: The column to sort the DataFrame by; The "leaderboard" column.
    top5_parameter: The values in the top-n that's returned.
    top: Amount of leaderboard rows. Based at 5.
    ascend: Set to False by default to return a descending leaderboard. Check True to return a rising leaderboard.
    xtra_lb_col: Set to True by default. Return both given columns (lboard_col and top5_parameter).
    ranked_idx: Set to True by default. Make the ranking's index start at 1.
    '''
    n=top
    if xtra_lb_col== True:
        ranking = df.sort_values(lboard_col, ascending = ascend)\
        .reset_index(drop=True).loc[:,[top5_parameter, lboard_col]][:n]
    else:
        ranking = df.sort_values(lboard_col, ascending = ascend).reset_index(drop=True)[top5_parameter][:n]
    
    if ranked_idx == True:
        ranking.index += 1
    
    return ranking

## 4.1 Exploring Capacity

To explore the capacity of solar farms, we'll need to create 2 pivot tables:
- One summing the total and average capacity for each region, divided by countries per year.
- Another one that does the same without years.

Let's create the pivot tables below:

In [ ]:
# A) Making a pivot table of the average and total capacity per region, per country, 
# divided to separate years -
capacity_per_year = global_solar.pivot_table(values='capacity_mw', index=['start_year', 'region', 'country'], \
                                            dropna=False, aggfunc={'mean', 'sum'}).dropna()
# Flattening the pivot table MultiIndex -
flat_TripleIdx(capacity_per_year,'start_year','region','country', reorder=True)

# -------------------------------------------------------------------------------------------------------------------------
# B) Making a pivot table of the average and total capacity per region, per country -
capacity_per_reg = global_solar.pivot_table(values='capacity_mw', index=['region', 'country'], \
                                            dropna=False, aggfunc={'mean', 'sum'}).dropna()

# Flattening the pivot table MultiIndex -
flat_TripleIdx(capacity_per_reg,'region','country', dub=True, reorder=True)

# -------------------------------------------------------------------------------------------------------------------------
# C) Displaying the pivot table -
print('\n\033[1m\x1b[4mDisplaying the DataFrames first 10 rows\x1b[0m:')
display_sided(capacity_per_year.head(10), capacity_per_reg.head(10),\
              titles=['Per Year, Divided to Regions, Countries (1st DF)', 'Per Region, Country (2st DF)'])

### 4.1.1 Average capacity comparisons

Now that the filtered pivot DFs are ready, let's plot the average capacity per region in a barplot, where each bar is divided to countries inside it:

In [ ]:
# Plotting a barplot showing the tota capacity per
fig = px.bar(capacity_per_reg.sort_values('mean', ascending = False), x='region', y='mean', height=700, width=975, \
             color='country', color_discrete_sequence= px.colors.sequential.Rainbow_r, opacity=0.85, \
             title='Average Capacity per Region, Divided by Countries', barmode='stack', \
            hover_name='country', hover_data=['country','sum'])

# Plot modification:
fig.update_layout(title_font_size=22, font_size=14, legend_title_font_size=16, legend_title='Country', \
                  xaxis_title="Region", yaxis_title="Average Capacity (MW)", \
                  uniformtext_mode='hide')
# Setting hovering spike lines -
fig.update_xaxes(showspikes=True, spikesnap="cursor", spikethickness=2)
fig.update_yaxes(showspikes=True, spikesnap="cursor", spikethickness=2)
fig.update_layout(spikedistance=1000, hoverdistance=100)
## Setting custom hovering data -
fig.update_traces(hovertemplate="<br>".join([
                      "<b>Country:</b> %{customdata[0]}",
                      '',
                      "Average Capacity (MW): %{y}",
                      "All-farms Total Capacity (MW): %{customdata[1]}",
                      "Region: %{x}"]))
# Show plot -
fig.show()

# -------------------------------------------------------------------------------------------------------------------------
# Creating some rankings -
highest_outpout_countries = (top_n(capacity_per_reg, 'mean', 'country', top=10))
lowest_outpout_countries = (top_n(capacity_per_reg, 'mean', 'country', top=10, ascend=True))
# Changing rankings columns' names for display -
highest_outpout_countries.columns = ['Country', 'Average MWs per Project']
lowest_outpout_countries.columns = ['Country', 'Average MWs per Project']

# Printing rankings -
print('\n\033[1m\x1b[4mThe top 10 countries with highest and lowest average energy output\x1b[0m:', end='')
display_sided(round(highest_outpout_countries, ndigits=2), round(lowest_outpout_countries, ndigits=2), \
              titles=['Highest Output', 'Lowest Output'])

#### ***Analyzing the regions***

As we can observe, the region with the highest average energy output is Asia, with an average of 5405 MWs per solar farm project, having more than doubled average energy output than each of the Europe and the other lowest energy output regions. Followed by Asia are the Middle East region (averagely 4370 Mws per a solar farm project), Africa (averagely 3752 MWs per a solar farm project), and Europe (averagely 2400 MWs per a solar farm project); These top 4 regions have a much higher average energy output than the other five regions, As all of the top regions have at least more than 3.5 times of any of these last regions. The region with the lowest average energy output is North America, with 252 MWs per each solar project.

If we ignore North Korea, which has only one solar facility and is bumping the average energy output for Asia drastically (Try clicking the country on the interactive plot's legend), we can see that The region that produces the highest energy output is actually the Middle East region, and that Asia is dropping to the third place below Africa without North Korea.

#### ***Analyzing the countries***

Zooming to the micro-level, the leading countries are Oman (averagely 1439.09 MWs per average solar project, Middle East), Kuwait (averagely 1265 MWs per solar project, Middle East), and Morocco (averagely 539.33 MWs per solar project, Africa). North Korea (Asia) average 2500 MWs, but that's just because it has only one solar farm overall. The countries with the absolute lowest energy output are Somalia and Sudan, tied at the first place with an average of 10 MWs per solar project, which is also minimum capacity allowed in Arab countries. Following them are a list of countries - as both Barbados, South Sudan, Madagascar, Liberia and Guinea-Bissau average a capacity of 20 MWs per solar project in the heavily tied second place.

Seems like the leading countries are tending to be usually Arab countries and the countries with the lowest average energy output are tending to be usually from third-world countries.

### 4.1.2 Mapping global energy output

We'll use the coordinates in the `global_solar` DF to map all solar farm projects on the world map so that we could see the global energy output of all solar farms visually:

In [ ]:
# Interpolating circle radius to given circle range -
rad = global_solar.capacity_mw.values.tolist()
m = intp.interp1d([1,max(rad)], [5,77.5])
circ_rads = m(rad)

# Plotting the energy
fig = px.density_mapbox(global_solar, lat='latitude', lon='longitude', zoom=0.75, radius=circ_rads, \
                        height=675, width=975, \
                        color_continuous_scale=px.colors.sequential.Hot, mapbox_style='carto-darkmatter',\
                        hover_data=['country','capacity_mw','region','city', 'local_area','major_area',\
                                    'project_name','operator','owner'])

# Plot modifications:
fig.update_layout(title='Global Energy Output', title_font_size=32,legend_title_font_size=22)              
# Setting custom hovering data -
fig.update_traces(hoverlabel_bgcolor='darkred', hoverlabel_bordercolor='gold',
    hovertemplate="<br>".join([
        "<b>Project:</b> %{customdata[6]}",
        'Operations are ran by %{customdata[7]}',
        'Owned by %{customdata[8]}',
        '',
        "Coordinates: %{lat},%{lon}",
        "Capacity (MW): %{customdata[1]}",
        "Country: %{customdata[0]}",
        "City: %{customdata[3]}",
        "Area (County|District): %{customdata[4]} | %{customdata[5]}",
        "Region: %{customdata[2]}"
    ]))

# Showing the density maplot -
fig.show()

We can see pretty strongly that Asia has the highest Energy output overall, and that solar farms are distributed pretty equally around east to central Asia, and in India as well. Europe is also pretty distributed with heavy energy output farms around mostly Italy, The UK, and western Europe (i.e. Spain and Portugal). On the other hand, solar farms on the Middle east aren't distributed so heavily and it looks like there are also less farms than the other regions we spoke about before, while this might be explained by that in this region each solar plant produces a bigger energy output than the regular solar farms around Asia and Europe.

***The leading energy output farms***

You could also notice a big red blur in North Australia that's located in the city of Elliot on Barkley Region county. It represents the Powell Creek solar farm which is the most productive solar farm **with an energy output of 20K MegaWatts**. After that farm, the leading farms globally are the Al Wusta Solar Plant in Oman (12.5k MWs), which is located in the wastelands at the south-west part of the country, and the Ladakh Solar park in India (10k MWs), which is located in the city of Leh in the Nubra county.

***The north is empty***

We can see that there are *barely* any solar farms in the north, to be specific - in Russia, the Norse countries (except Denmark), Greenland, Iceland and North Canada. That might be because these regions don't have enough sun light, light hours or heat to be prolific enough for solar energy production. On the furthest contrary side, **Finland is an exception as it has the northernmost 2 solar farms in the entire world**, the Lapua and Palloneva solar farms, which are producing 80 MWs and 500 MWs, orderly. **Lapua solar farm is the northernmost farm of all**, located in the city of Lapua at the Seinäjoki sub-region.

***The higher the sun shine***

There is also another interesting fact that requires attention, **the majority of the solar farms are located on the coast, or in countries that are close to the coast** - this can be seen perfectly in Africa, where the solar farms who produce the highest energy signatures are the ones close to the coast. Also note that each of East Asia, East Australia, India and both the Americas shows the same supporting evidence to that assumption.

## 4.2 Exploring amounts of solar farms throughout the years

In order to explore the change in the amounts of solar farms throughout the years, we'll create a pivot table that shows the amount of projects in every country, region throughout the years:

In [ ]:
# Creating a pivot table showing the amount of projects started in each recorded year -
proj_p_y = global_solar.pivot_table(values='project_name', \
                                            index=['start_year', 'region', 'country'], aggfunc='count')

# Flatening MultiIndex -
flat_TripleIdx(proj_p_y, 'start_year', 'region', 'country', reorder=True)

# Displaying the pivot table -
proj_p_y

The data we've created represents each country's solar farms amount per a recorded year. 

Now that the data is ready, let's plot the growth rates in solar farms for each region:

In [ ]:
# Plotting the cumulative amounts of solar farms in each region -
fig = px.ecdf(proj_p_y, x='start_year', y='project_name', color='region', \
             color_discrete_sequence= px.colors.sequential.Plasma_r, height=745, width=1000, \
             range_x=(2006.5,2028), ecdfnorm=None, marginal='box', markers=True, \
             hover_data=['region', 'start_year', 'project_name'])

# Plot modifications:
fig.update_layout(title='Total number of solar farms per Region', title_font_size=22,\
                  legend_title='Region:', legend_title_font_size=22, legend_font_size=12,
                  xaxis_title="Year", yaxis_title="Amount of Solar Farms", \
                  font_size=16, uniformtext_mode='hide')
# Setting hovering spike lines -
fig.update_xaxes(showspikes=True, spikethickness=2, spikesnap="cursor", spikemode="across")
fig.update_yaxes(showspikes=True, spikethickness=2, spikemode="across")
fig.update_layout(spikedistance=500, hoverdistance=25)
# Setting custom hovering data -
fig.update_traces(hoverlabel_bgcolor='gold', hoverlabel_bordercolor='rebeccapurple',
                  hovertemplate="<br>".join([
                      "<b>Region:</b> %{customdata[0]}",
                      '',
                      "Amount of farms: %{y}",
                      "Year: %{x}"]))
# Adding delimiting line & area to the plot
# to mark future forecasts according to data on future projects -
fig.add_vline(x=2023, line_width=2, line_dash="dash", line_color="rebeccapurple")
fig.add_vrect(x0=2023, x1=2030, line_width=0, fillcolor="rebeccapurple", opacity=0.15, \
             annotation_text="Forecasts", annotation_position="top left", \
             annotation=dict(font_size=18),)

# Showing plot -
fig.show()

*Plot Note: The data on the faded purple frame are on the right represents known future projects that are expected to start operating in the following years.*

Looking at the marginal boxplots above the graph, we can see that all of the countries are equally distributed, while the peak of globally growth has been reached to in the year of 2019 - since that year, growth rate seems to have been faded in all regions equally. That global decline in new solar numbers could possibly be explained by the global Coronavirus shutdown in 2020 to the beginning of 2021, that might have shelved, canceled or delayed a lot of projects from starting to run.

We can clearly see that Asia has the highest number of solar farms, by a supreme distance over all others regions - as since 2013, a huge growth rate trendline has started for the region, growing by at least 300 solar farms year to year up until the 2019 peak. The runner up is surprisingly North America, which might be really low on total energy output, but are opening more a more solar plants throughout the years. Central America and the Caribbean is the region that had the weakest growth rate with the lowest number of solar projects 42 in the current year (2022) while the following regions are the Eurasia with 42 solar farms, and the Middle East with 64 farms in 2022.

## 4.3 Top ownership companies and individuals

We'll check which companies own the most solar farms globally to determine has the biggest control over the solar market.

Let's begin by creating a pivot table for companies who own the most solar farm projects, and then create another one for finding the top 10 owners:

In [ ]:
# Creating a pivot table that shows the companies who own the most solar farms globally -
owner_ranks = global_solar.pivot_table(values='project_name',index='owner', aggfunc='count')

# Flattening the index and sorting the values as descending -
owner_ranks = flat_Idx(owner_ranks,'owner', reorder=True).sort_values(by='project_name', ascending=False)

# -------------------------------------------------------------------------------------------------------------------------
# Finding the top 10 owners who are not unknown -
top10_owners = top_n(owner_ranks.query('owner != "Unknown"'),'project_name','owner',10)

# -------------------------------------------------------------------------------------------------------------------------
# Displaying DFs -
print('\n\033[1m\x1b[4mDisplaying the "Owners Ranking" and the top 10 ranked owners (The leading 10 owners)\x1b[0m:', end='')
display_sided(owner_ranks.head(10), top10_owners.head(10), \
             titles=['The `owner_ranks` DF', 'The `top10_owner` DF'])

Now that the DFs are created. we'll plot a barplot representing the globally leading 10 ownerships:

In [ ]:
# Plotting the leading 10 owners in the industry -
fig = px.bar(top10_owners,x='owner',y='project_name', color='owner', \
      title='Companies who own the most solar farms', height=600, width=950,\
            color_discrete_sequence= px.colors.sequential.solar_r)

# Plot modifications:
fig.update_layout(title='Total solar farms for the Top 10 Ownerships', title_font_size=22,\
                  legend_title='Ownership:', legend_title_font_size=16, legend_font_size=12,
                  xaxis_title="Ownership Company", yaxis_title="Amount", \
                  font_size=14, uniformtext_mode='hide')
fig.update_xaxes(tickangle = 25)
# Setting hovering spike lines -
fig.update_yaxes(showspikes=True, spikethickness=2, spikemode="across")
fig.update_layout(spikedistance=500, hoverdistance=25)
# Setting custom hovering data -
fig.update_traces(hoverlabel_bgcolor='gold', hoverlabel_bordercolor='rebeccapurple',
                  hovertemplate="<br>".join([
                      "<b>Owner:</b> %{x}",
                      "<b>Owned solar farms:</b> %{y}"]))       
# Showing plot -
fig.show()

# -------------------------------------------------------------------------------------------------------------------------
# Calculating some stats -
unique_owners = len(owner_ranks['owner'])
more_than_10 = len(owner_ranks[(owner_ranks['owner'] != 'Unknown') & (owner_ranks['project_name'] > 10)])
less_than_10 = len(owner_ranks[(owner_ranks['owner'] != 'Unknown') & (owner_ranks['project_name'] < 10)])

# Printing stats -
print(f"\033[1m\x1b[4mSome stats\x1b[0m:\n\n\
There are \033[1m\x1b[4m{unique_owners} ownership companies\x1b[0m around the solar market.\n\
Companies who have more than 10 farms: \033[4m\x1b[31m{more_than_10} ownership companies\x1b[0m.\n\
Companies who have less than 10 farms: {less_than_10} ownership companies.")

The company who owns the most solar farms around the world is NextEra Energy, which has 140 solar farms under it's command, more than 2 times of the other 3rd to 10th places in the ranking, and about 1.7 times the 2nd place amount; NextEra Energy are taking a much bigger part of the solar energy solution to the oil and petrol recession than the entire industry, pretty much controlling the Solar Energy market.

The following leading ownerships companies were the CECEP Solar Energy Technology company, having 82 solar farms under it's belt, the CGN Solar Energy Development company, that possess 70 solar farms globally, and Solatio Energia who owns 62 solar farms. The rest of the leading 10 have pretty similar amounts that range from Adani Green Energy's 56 solar farms to Duke Energy's 48 farms.

Be noted that right now, out of the 3453 known ownership companies or individuals, Only 112 owners possess more than 10 solar farms overall, i.e. there are 3323 ownerships that are under-represented in the solar market, while the leading companies control the majority. There's a cartel union in the solar farms market, where the top companies control a much bigger bigger share than the rest of the small ones.


------------------
------------------

# 5. General Conclusion

At the first time we observed the data during the Data Exploration section, we've noticed some problems:
- All columns' names were capitalized, while some possessed really long names.
- There were 15 columns with missing values, out of which 8 columns are clearly fine to ignore. The columns who required the special attention were:
    - **`Phase Name`**
    - **`City`**
    - **`Major area (prefecture, district)`**
    - **`Operator`**
    - **`Local area (taluk, county)`**
    - **`Owner`**
    - **`State/Province`**
- In the 'About' section of the database, the author of the data advised to check the consistency across the `'Country` column trackers, although stating it should have been ordered.

## **5.1 Data Preprocessing**

We started preprocessing the data in the following ways:

***1. Fixing columns names:***

We started by changing all the columns' names to short and `'snake_cased'` names, so they'll be easier to work with.

***2. Wrong dtypes***

We've fixed wrong columns' data types:
    - The `'start_year'` and `'retired_year'` columns dtype has been changed from `float64` to `Int64`.
    - The `'country'` and `'region'` `'status'` columns dtype has been changed from `object` to `category`.

***3. Validating data on the `'country'` column***

We've noticed that the `'country'` column was sorted alphabetically only partly on the first rows, but on the plus side there are no misspelling or partial duplicated country names. To fix this we have sorted the data by the `'country'` column and resetted the index.

***4. Handling the missing values***

To decide how should we handle missing values, we've first explored all missing values for possible connections using some 'missingno' missing values plots, and we've found that overall there aren't any fully based connections between missing values, except:
- The `'start_year'` and `'retired_year'` columns had a full correlation of 1, which makes sense because the database author has stated that the starting year for projects who got canceled or shelved was left missing, so projects who got canceled cannot be retired.
- Using the matrix plot, we've seen that missing values tend to look differently in different rows groups locations on the table, hinting there might be some connection between the missing values while looking at the columns for rows of similar countries. We've also seen that some missing rows of the the owner and operator columns seems to have full values on their correlating local name columns.

***4.1 Afterwards, we've started handling the 7 columns that we've decided that should be handled in prior stages:***4.1
1. I've decided to fill the missing values in the location column with their coordinates values. We've created the `'coordinates'` column, that created a string dtyped column containing the latitude and longitude in the accepted format of it. After that I've created a reversed mapping function that is used for mapping the coordinates exact addresses. We used that function to map the missing values in the following columns, while the rest of the missing values were filled with the value 'Unavailable':
    - We mapped and filled 252 locations out of the 647 missing values in the `state_prov` column.
    - We mapped and filled 582 locations out of the 3920 missing values in the `major_area` column.
    - We mapped and filled 1309 locations out of the 3207 missing values in the `local_area` column.
    - We mapped and filled 1618 exactly accurate locations out of the 1964 missing values with an `'exact'` location accuracy, and from a total amount of 4422 missing values in the `city` column itself (That's because mapping the city of approx.. accurate coordinates will not be prolific).


2. For missing values with local names on the 'local name' columns, we've translated local names to fill values, filling the remaining missing values with the value 'Unknown:
    - **`operator`** - 7 values were filled out of 3290.
    - **`owner`** - One value was filled out of 840.


3. We've managed to fill 28 rows out of the 5350 missing values in the `phase_name` column using the non-encrypted other IDs in the `other_unit_phase_id` column. Since missing values take too much big of a share from the column, we left the rest of the values as they were.

## **5.2 The Analysis and Data Visualization**

We've explored a few areas in that section, as my top conclusions were as follows.

### ***Conclusions regarding capacity output:***

Asia is the region that produces the highest energy output in the entire world, but without North Korea, it's dropping to the 3rd place, symbolizing that it constitutes a huge part of Asia leading the solar market output. The Middle East region produces a lot of energy, and **although it has less solar farms, each farm is producing much more solar energy averagely than in any other region**. Solar farm located in Europe have the exact opposite story, as they tend to have a smaller capacity than most solar farms in the big regions, but are compensating mostly in the numbers, as they have a lot of solar farms distributed mostly in Greece, The UK, Italy and West Europe (a.k.a Spain and Portugal) shores, majorly. solar farms located in Africa have a pretty high average capacity and great sunlight conditions, making their relatively small amount of solar farms powerful enough to compete with the big regions, although the vast majority of the solar farms in the region are located near the edges of the continent, while most of central Africa seems to lack solar farms.

There seems to be a big effect on climate and location on the successful output of a solar farm, as the countries who produce the most solar energy are usually Arab countries, that are located in hot climate areas like Africa or close to the coast, where they get a lot more sunlight than shore-less countries. A special notation should go to Australia, who has a huge amount of low capacity solar plants in the east of the continent, but are compensating it with the Power Creek solar farm, which is the largest solar energy producer on the globe.

There is a black hole lacking energy output in the north side of the globe, that seems to result from the lack of the conditions discussed in the above paragraph - having a cold climate, low amount of sunlight and smaller daytime. On the other hand, the country of Finland specifically, which is known for having a surprisingly great climate *although* it's geographical location, is the northernmost place in the world for a solar farm to appear.

#### **To create an exact profile:**
- The countries with ***the highest*** average solar energy production are tending to be usually Arab countries, with great access to heat and sunlight, located near the coast or in a country close to the coast.
- The countries with ***the lowest*** average energy production are tending to be usually poor third-world countries, located far from shores, or in the north side of the globe.

### ***Conclusions regarding the growth in the amounts of solar energy production:***

It seems like even though the solar farms market has started pretty low, it has gained popularity in the beginning of the last decade, around 2012-2013, while Asian countries have embraced the trend much faster than countries from all other regions, even though we could clearly see a huge drop in numbers of new solar farms since 2020, which could easily be explained by the Coronavirus pandemic global shutdown which has affected the entire world, that was probably the reason it was hard for business owners to open new farms.

Asia has the highest growth rate around all regions, having at least 300 new solar facilities built over every year, up until the global shutdown in 2020, Asia tallied a total of 3461 solar farms in 2022. Meanwhile, we can see that although North America is the region that produces the lowest solar energy around the world, it actually ignored the shutdown in 2020 and has started to rapidly grow in amounts. The region that has the lowest amount of solar farms in 2022 is Central America and The Caribbeans with 42 farms, and it also has the lowest growth rate. The Middle East region currently has 64 solar plants in 2022.

### ***Conclusions regarding ownerships in the solar energy market:***

There seems to be a carte union in the global solar energy market, as while there are 3453 recorded ownerships of solar farms globally, only 112 companies out of them own more than 10 farms, and the top 4 owners each hold more than 62 farms each. The company that controls the biggest share of the solar market is the NextEra Energy company with an owned quantity of 140 solar farms.

## 5.3 Final words

While there might have been a slowdown in the past 2 years because of the global pandemic, the future is looking bright and sunny, as more and more countries are starting to invest in solar energy as an alternate main energy resource in preparation for the eradication petrol, while the leaders of the revolution are Asian, Middle Eastern and African countries and the pioneering NextEra company, along with the rest of the solar farms investors. As when the growth rate will return to it's normal self, in a few years we might be using the sun as our main energy source and make the Earth greener.